In [1]:
import netCDF4 as nc
import numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import os
import cartopy.feature as cfeature
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
#import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cf
import mplotutils as mpu # helper functions for cartopy and matplotlib
#import regionmask
from func_plots import *
from func_stats import *

In [2]:
# Regional plot with coastline and country borders
def format_axes(axes):
    for i, ax in enumerate(axes):
        #ax.set_title(plot_names[i])
        ax.coastlines(resolution='50m', linewidth=0.45)
        # ax.add_feature(cf.BORDERS, linewidth=0.3)
        ax.set_extent([lonmin, lonmax, latmin, latmax], crs=data_proj)
        ax.gridlines(draw_labels=False, linewidth=0.3, color="gray", xlocs=range(-180, 180, 10), ylocs=range(-90, 90, 10))

def add_headers(fig, *, row_headers=None, col_headers=None, row_pad=1, col_pad=4, rotate_row_headers=True, **text_kwargs):
    # Based on https://stackoverflow.com/a/25814386
    axes = fig.get_axes()

    for ax in axes:
        sbs = ax.get_subplotspec()

        # Putting headers on cols
        if (col_headers is not None) and sbs.is_first_row():
            ax.annotate(
                col_headers[sbs.colspan.start],
                xy=(0.5, 1),
                xytext=(0, col_pad),
                xycoords="axes fraction",
                textcoords="offset points",
                ha="center",
                va="baseline",
                **text_kwargs,
            )

        # Putting headers on rows
        if (row_headers is not None) and sbs.is_first_col():
            ax.annotate(
                row_headers[sbs.rowspan.start],
                xy=(0, 0.5),
                xytext=(-ax.yaxis.labelpad - row_pad, 0),
                xycoords=ax.transAxes, # ax.yaxis.label replaced by ax.transAxes
                textcoords="offset points",
                ha="center",
                va="center", # vertical even after rotation
                rotation=rotate_row_headers * 90,
                **text_kwargs,
            )

lonmin, lonmax, latmin, latmax = [-11, 37, 35, 70.5]                   # window for plotting
set_lonmin, set_lonmax, set_latmin, set_latmax = [-35, 65, 30, 72.6]   # subset the data to get sensible vmin and vmax for the colorbar
data_proj = ccrs.PlateCarree()
map_proj = ccrs.LambertConformal(central_longitude=15) # for regional maps

In [3]:
filepath = '/landclim2/yiyaoy/COSMO-CLM2-simulations/timmean/clm5.0_eur0.5_control_h0_2020-2024.nc_timmean'
with nc.Dataset(filepath) as ds:
    # the [:] slice gives you a NumPy array right away
    lat = ds.variables['lat'][:]
    lon = ds.variables['lon'][:]

# vectorized shift from [0,360] → [-180,180]
lon = (lon + 180) % 360 - 180

In [4]:
def get_data_mean_for_plot(var, lat, lon):
    """
    Load annual, JJA and MAM means for control, only_forest_broadleaf and all_grass
    into one xarray.Dataset keyed by:
      control, control_JJA, control_MAM,
      only_forest_broadleaf, only_forest_broadleaf_JJA, only_forest_broadleaf_MAM,
      all_grass, all_grass_JJA, all_grass_MAM
    """
    scenarios = {
        "control":                  "control/clm5",
        "only_forest_broadleaf":    "only_forest_broadleaf/clm5",
        "all_grass":                "all_grass/clm5",
    }
    seasons = {
        "":     "timmean",
        "_JJA": "seasonal/timmean",
        "_MAM": "seasonal/timmean",
    }

    template = (
        "/landclim2/yiyaoy/COSMO-CLM2-simulations/{folder}/post-processing/{proc}/"
        "mergetime/clm5.0_eur0.5_{scen}_h0_2025-2059{suffix}.nc_timmean_timmean"
    )

    data_vars = {}
    for scen, folder in scenarios.items():
        for suffix, proc in seasons.items():
            key  = f"{scen}{suffix}"
            path = template.format(folder=folder, proc=proc, scen=scen, suffix=suffix)
            
            # load, squeeze, mask >1e9, wrap in DataArray
            arr = nc.Dataset(path).variables[var][:]
            arr = np.squeeze(np.array(arr))
            arr[arr > 1e9] = np.nan
            da = xr.DataArray(arr, coords={"y": lat, "x": lon}, dims=["y", "x"])
            
            data_vars[key] = da

    return xr.Dataset(data_vars)

In [5]:
SWup_M = get_data_mean_for_plot('SWup', lat, lon)
SWdown_M = get_data_mean_for_plot('SWdown', lat, lon)
SWnet_M = SWdown_M - SWup_M

In [ ]:
# 1) contour settings
levels_dict = {
    "seq0": np.array([10, 15, 20, 25, 30, 35, 40]),
    "seq1": np.array([30, 60, 90, 120, 150, 180, 210]),
    "seq2": np.array([10, 40, 70, 100, 130, 160, 190]),
    "div": np.array([-16, -8, -4, -2, -1, 1, 2, 4, 8, 16])
}
ticks_dict = {
    "seq0": ["10", "15", "20", "25", "30", "35", "40"],
    "seq1": ["30", "60", "90", "120", "150", "180", "210"],
    "seq2": ["10", "40", "70", "100", "130", "160", "190"],
    "div": ['\N{MINUS SIGN}16', '\N{MINUS SIGN}8', '\N{MINUS SIGN}4', '\N{MINUS SIGN}2', '\N{MINUS SIGN}1', '+1', '+2', '+4', '+8', '+16']
}
cmaps = {"seq0": "hot_r", "seq1": "hot_r", "seq2": "hot_r", "div": "RdBu_r"}
unit  = "$\\mathregular{W/m^2}$"
vmin, vmax, extend = 0, 42, "both"


# 2) pack your three xarray‐datasets
datasets = {
    "up": SWup_M,
    "down": SWdown_M,
    "net": SWnet_M,
}

# 3) define each subplot:
#    (which dataset key, high_key, low_key, row, col, levels‐type, title‐stem, panel‐letter)
panels = [
    ("up", "control",             None,         0, 0, "seq0", "Ctl: $\mathregular{SW_{up,yearM}}$",             "a"),
    ("up", "only_forest_broadleaf","control",   0, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{up,yearM}}$",   "b"),
    ("up", "all_grass",            "control",   0, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{up,yearM}}$",   "c"),
    ("up", "only_forest_broadleaf","all_grass",  0, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{up,yearM}}$",   "d"),

    ("down", "control",              None,         1, 0, "seq1", "Ctl: $\mathregular{SW_{down,yearM}}$",             "e"),
    ("down", "only_forest_broadleaf","control",   1, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{down,yearM}}$",   "f"),
    ("down", "all_grass",            "control",   1, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{down,yearM}}$",   "g"),
    ("down", "only_forest_broadleaf","all_grass",  1, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{down,yearM}}$",   "h"),

    ("net", "control",              None,         2, 0, "seq2", "Ctl: $\mathregular{SW_{net,yearM}}$",             "i"),
    ("net", "only_forest_broadleaf","control",   2, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{net,yearM}}$",   "j"),
    ("net", "all_grass",            "control",   2, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{net,yearM}}$",   "k"),
    ("net", "only_forest_broadleaf","all_grass",  2, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{net,yearM}}$",   "l"),
]

# 4) build the figure
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 5) loop through panels
for dset_key, hi, lo, r, c, lvl_type, title_stem, letter in panels:
    ax     = axes[r, c]
    ds     = datasets[dset_key]
    arr    = ds[hi] if lo is None else (ds[hi] - ds[lo])
    levels = levels_dict[lvl_type]
    ticks  = ticks_dict[lvl_type]
    cmap   = cmaps[lvl_type]

    h = arr.plot(
        ax=ax,
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )
    ax.set_title(f"{title_stem} ({unit})")
    ax.add_feature(cfeature.OCEAN, color="whitesmoke")

    # always draw colorbar on every panel here (remove the if-col==3 test if you want all)
    cbar = mpu.colorbar(
        h, ax,
        orientation="vertical",
        size=0.04,
        extend=extend,
        pad=0.05,
        ticks=levels
    )
    cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        transform=ax.transAxes,
        fontsize=10, fontweight="bold"
    )

In [ ]:
# 1) contour settings
levels_dict = {
    "seq0": np.array([10, 15, 20, 25, 30, 35, 40]),
    "seq1": np.array([ 60, 90, 120, 150, 180, 210, 240]),
    "seq2": np.array([40, 70, 100, 130, 160, 190, 220]),
    "div": np.array([-16, -8, -4, -2, -1, 1, 2, 4, 8, 16])
}
ticks_dict = {
    "seq0": ["10", "15", "20", "25", "30", "35", "40"],
    "seq1": ["60", "90", "120", "150", "180", "210", "240"],
    "seq2": ["40", "70", "100", "130", "160", "190", "220"],
    "div": ['\N{MINUS SIGN}16', '\N{MINUS SIGN}8', '\N{MINUS SIGN}4', '\N{MINUS SIGN}2', '\N{MINUS SIGN}1', '+1', '+2', '+4', '+8', '+16']
}
cmaps = {"seq0": "hot_r", "seq1": "hot_r", "seq2": "hot_r", "div": "RdBu_r"}
unit  = "$\\mathregular{W/m^2}$"
vmin, vmax, extend = 0, 42, "both"


# 2) pack your three xarray‐datasets
datasets = {
    "up": SWup_M,
    "down": SWdown_M,
    "net": SWnet_M,
}

# 3) define each subplot:
#    (which dataset key, high_key, low_key, row, col, levels‐type, title‐stem, panel‐letter)
panels = [
    ("up", "control_JJA",             None,         0, 0, "seq0", "Ctl: $\mathregular{SW_{up,jjaM}}$",             "a"),
    ("up", "only_forest_broadleaf_JJA","control_JJA",   0, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{up,jjaM}}$",   "b"),
    ("up", "all_grass_JJA",            "control_JJA",   0, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{up,jjaM}}$",   "c"),
    ("up", "only_forest_broadleaf_JJA","all_grass_JJA",  0, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{up,jjaM}}$",   "d"),

    ("down", "control_JJA",              None,         1, 0, "seq1", "Ctl: $\mathregular{SW_{down,jjaM}}$",             "e"),
    ("down", "only_forest_broadleaf_JJA","control_JJA",   1, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{down,jjaM}}$",   "f"),
    ("down", "all_grass_JJA",            "control_JJA",   1, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{down,jjaM}}$",   "g"),
    ("down", "only_forest_broadleaf_JJA","all_grass_JJA",  1, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{down,jjaM}}$",   "h"),

    ("net", "control_JJA",              None,         2, 0, "seq2", "Ctl: $\mathregular{SW_{net,jjaM}}$",             "i"),
    ("net", "only_forest_broadleaf_JJA","control_JJA",   2, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{net,jjaM}}$",   "j"),
    ("net", "all_grass_JJA",            "control_JJA",   2, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{net,jjaM}}$",   "k"),
    ("net", "only_forest_broadleaf_JJA","all_grass_JJA",  2, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{net,jjaM}}$",   "l"),
]

# 4) build the figure
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 5) loop through panels
for dset_key, hi, lo, r, c, lvl_type, title_stem, letter in panels:
    ax     = axes[r, c]
    ds     = datasets[dset_key]
    arr    = ds[hi] if lo is None else (ds[hi] - ds[lo])
    levels = levels_dict[lvl_type]
    ticks  = ticks_dict[lvl_type]
    cmap   = cmaps[lvl_type]

    h = arr.plot(
        ax=ax,
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )
    ax.set_title(f"{title_stem} ({unit})")
    ax.add_feature(cfeature.OCEAN, color="whitesmoke")

    # always draw colorbar on every panel here (remove the if-col==3 test if you want all)
    cbar = mpu.colorbar(
        h, ax,
        orientation="vertical",
        size=0.04,
        extend=extend,
        pad=0.05,
        ticks=levels
    )
    cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        transform=ax.transAxes,
        fontsize=10, fontweight="bold"
    )

In [ ]:
# 1) contour settings
levels_dict = {
    "seq0": np.array([10, 15, 20, 25, 30, 35, 40]),
    "seq1": np.array([30, 60, 90, 120, 150, 180, 210]),
    "seq2": np.array([10, 40, 70, 100, 130, 160, 190]),
    "div": np.array([-16, -8, -4, -2, -1, 1, 2, 4, 8, 16])
}
ticks_dict = {
    "seq0": ["10", "15", "20", "25", "30", "35", "40"],
    "seq1": ["30", "60", "90", "120", "150", "180", "210"],
    "seq2": ["10", "40", "70", "100", "130", "160", "190"],
    "div": ['\N{MINUS SIGN}16', '\N{MINUS SIGN}8', '\N{MINUS SIGN}4', '\N{MINUS SIGN}2', '\N{MINUS SIGN}1', '+1', '+2', '+4', '+8', '+16']
}
cmaps = {"seq0": "hot_r", "seq1": "hot_r", "seq2": "hot_r", "div": "RdBu_r"}
unit  = "$\\mathregular{W/m^2}$"
vmin, vmax, extend = 0, 42, "both"


# 2) pack your three xarray‐datasets
datasets = {
    "up": SWup_M,
    "down": SWdown_M,
    "net": SWnet_M,
}

# 3) define each subplot:
#    (which dataset key, high_key, low_key, row, col, levels‐type, title‐stem, panel‐letter)
panels = [
    ("up", "control_MAM",             None,         0, 0, "seq0", "Ctl: $\mathregular{SW_{up,mamM}}$",             "a"),
    ("up", "only_forest_broadleaf_MAM","control_MAM",   0, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{up,mamM}}$",   "b"),
    ("up", "all_grass_MAM",            "control_MAM",   0, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{up,mamM}}$",   "c"),
    ("up", "only_forest_broadleaf_MAM","all_grass_MAM",  0, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{up,mamM}}$",   "d"),

    ("down", "control_MAM",              None,         1, 0, "seq1", "Ctl: $\mathregular{SW_{down,mamM}}$",             "e"),
    ("down", "only_forest_broadleaf_MAM","control_MAM",   1, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{down,mamM}}$",   "f"),
    ("down", "all_grass_MAM",            "control_MAM",   1, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{down,mamM}}$",   "g"),
    ("down", "only_forest_broadleaf_MAM","all_grass_MAM",  1, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{down,mamM}}$",   "h"),

    ("net", "control_MAM",              None,         2, 0, "seq2", "Ctl: $\mathregular{SW_{net,mamM}}$",             "i"),
    ("net", "only_forest_broadleaf_MAM","control_MAM",   2, 1, "div", "Brd–Ctl: $\\Delta$$\mathregular{SW_{net,mamM}}$",   "j"),
    ("net", "all_grass_MAM",            "control_MAM",   2, 2, "div", "Def–Ctl: $\\Delta$$\mathregular{SW_{net,mamM}}$",   "k"),
    ("net", "only_forest_broadleaf_MAM","all_grass_MAM",  2, 3, "div", "Brd–Def: $\\Delta$$\mathregular{SW_{net,mamM}}$",   "l"),
]

# 4) build the figure
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 5) loop through panels
for dset_key, hi, lo, r, c, lvl_type, title_stem, letter in panels:
    ax     = axes[r, c]
    ds     = datasets[dset_key]
    arr    = ds[hi] if lo is None else (ds[hi] - ds[lo])
    levels = levels_dict[lvl_type]
    ticks  = ticks_dict[lvl_type]
    cmap   = cmaps[lvl_type]

    h = arr.plot(
        ax=ax,
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )
    ax.set_title(f"{title_stem} ({unit})")
    ax.add_feature(cfeature.OCEAN, color="whitesmoke")

    # always draw colorbar on every panel here (remove the if-col==3 test if you want all)
    cbar = mpu.colorbar(
        h, ax,
        orientation="vertical",
        size=0.04,
        extend=extend,
        pad=0.05,
        ticks=levels
    )
    cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        transform=ax.transAxes,
        fontsize=10, fontweight="bold"
    )